### 贝叶斯拼写检查器


### 求解：argmaxc P(c|w) -> argmaxc P(w|c) P(c) / P(w) ###
* P(c), 文章中出现一个正确拼写词 c 的概率, 也就是说, 在英语文章中, c 出现的概率有多大
* P(w|c), 在用户想键入 c 的情况下敲成 w 的概率. 因为这个是代表用户会以多大的概率把 c 敲错成 w
* argmaxc, 用来枚举所有可能的 c 并且选取概率最大的

In [14]:
import re, collections

# 把语料中的单词全部抽取出来, 转成小写, 并且去除单词中间的特殊符号
def words(text):
    return re.findall('[a-z]+',text.lower())

# 统计词频
# 要是遇到我们从来没有过见过的新词怎么办. 假如说一个词拼写完全正确, 但是语料库中没有包含这个词, 从而这个词也永远不会出现在训练集中. 于是, 我们就要返回出现这个词的概率是0. 这个情况不太妙, 因为概率为0这个代表了这个事件绝对不可能发生, 而在我们的概率模型中, 我们期望用一个很小的概率来代表这种情况. lambda: 1
def train(features):
    model = collections.defaultdict(lambda : 1) # 设定字典默认值，如果没有key，返回1
    for f in features:
        model[f] += 1
    return model

file = open('/Users/joey/Documents/PycharmProjects/mywork/bayesian_algorithm/big.txt').read()
word_file = words(file)
NWORDS = train(word_file)


###  编辑距离: ###
两个词之间的编辑距离定义为使用了几次插入(在词中插入一个单字母), 删除(删除一个单字母), 交换(交换相邻两个字母), 替换(把一个字母换成另一个)的操作从一个词变到另一个词.
做了多少次操作就是距离多少

In [90]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
# word[0:0] == ''
# 编辑距离
#返回所有与单词 w 编辑距离为 1 的集合
def edits1(word):
    n = len(word)
    return set([word[0:i] + word[i+1:] for i in range(n)] +    # deletion
            [word[0:i] + word[i+1] + word[i] + word[i+2:] for i in range(n-1)] +     # transposition
            [word[0:i] + c + word[i+1:] for i in range(n) for c in alphabet] +      # alteration
            [word[0:i] + c + word[i:] for i in range(n+1) for c in alphabet])        # insertion

In [48]:
a = edits1('something')
a

{'aomething',
 'asomething',
 'bomething',
 'bsomething',
 'comething',
 'csomething',
 'domething',
 'dsomething',
 'eomething',
 'esomething',
 'fomething',
 'fsomething',
 'gomething',
 'gsomething',
 'homething',
 'hsomething',
 'iomething',
 'isomething',
 'jomething',
 'jsomething',
 'komething',
 'ksomething',
 'lomething',
 'lsomething',
 'momething',
 'msomething',
 'nomething',
 'nsomething',
 'omething',
 'oomething',
 'osmething',
 'osomething',
 'pomething',
 'psomething',
 'qomething',
 'qsomething',
 'romething',
 'rsomething',
 'samething',
 'saomething',
 'sbmething',
 'sbomething',
 'scmething',
 'scomething',
 'sdmething',
 'sdomething',
 'semething',
 'seomething',
 'sfmething',
 'sfomething',
 'sgmething',
 'sgomething',
 'shmething',
 'shomething',
 'simething',
 'siomething',
 'sjmething',
 'sjomething',
 'skmething',
 'skomething',
 'slmething',
 'slomething',
 'smething',
 'smmething',
 'smoething',
 'smomething',
 'snmething',
 'snomething',
 'soaething',
 'so

In [28]:
#返回所有与单词 w 编辑距离为 2 的集合
#在这些编辑距离小于2的词中间, 只把那些正确的词作为候选词
#在编辑距离为1的基础上，再做一次，就是距离为2
def know_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)
b = know_edits2('omg')
b

{'am',
 'ame',
 'ami',
 'amp',
 'amy',
 'aug',
 'bag',
 'beg',
 'big',
 'bog',
 'bomb',
 'bug',
 'cm',
 'cms',
 'com',
 'coma',
 'comb',
 'come',
 'cong',
 'dig',
 'dog',
 'dome',
 'domo',
 'dug',
 'eg',
 'egg',
 'em',
 'fig',
 'fog',
 'g',
 'gag',
 'gig',
 'go',
 'gong',
 'hm',
 'home',
 'hug',
 'imp',
 'ing',
 'jug',
 'keg',
 'km',
 'leg',
 'log',
 'lome',
 'long',
 'm',
 'ma',
 'mb',
 'md',
 'me',
 'meg',
 'mi',
 'mm',
 'mmm',
 'mo',
 'mob',
 'mon',
 'mop',
 'mot',
 'mr',
 'ms',
 'mt',
 'mug',
 'my',
 'nag',
 'nm',
 'nom',
 'noma',
 'o',
 'oak',
 'oar',
 'obs',
 'oct',
 'odd',
 'ode',
 'oe',
 'of',
 'off',
 'oft',
 'ogg',
 'oh',
 'oho',
 'oil',
 'ok',
 'oka',
 'old',
 'ole',
 'olga',
 'om',
 'omen',
 'omit',
 'omne',
 'on',
 'one',
 'ong',
 'ont',
 'oo',
 'ooh',
 'op',
 'opt',
 'or',
 'orb',
 'ore',
 'org',
 'os',
 'ou',
 'ouh',
 'oui',
 'our',
 'out',
 'ovo',
 'owe',
 'owl',
 'own',
 'ox',
 'oz',
 'peg',
 'pg',
 'pig',
 'pm',
 'pmb',
 'pomp',
 'rag',
 'rome',
 'rug',
 'sg',
 'some'

与 something 编辑距离为2的单词居然达到了 114,324 个
优化:在这些编辑距离小于2的词中间, 只把那些正确的词作为候选词,只能返回 3 个单词: ‘smoothing’, ‘something’ 和 ‘soothing’


In [30]:
def know(words):
    return set(w for w in words if w in NWORDS)
c = know('omg')
c

{'g', 'm', 'o'}

正常来说把一个元音拼成另一个的概率要大于辅音 (因为人常常把 hello 打成 hallo 这样);
把单词的第一个字母拼错的概率会相对小, 等等.但是为了简单起见, 选择了一个简单的方法:
编辑距离为1的正确单词比编辑距离为2的优先级高, 而编辑距离为0的正确单词优先级比编辑距离为1的高.

In [91]:
# 如果known(set)非空, candidate 就会选取这个集合, 而不继续计算后面的

# 集合里的值要通过函数传入逐项读取
def correct(word):
    candidates = know([word]) or know(edits1(word)) or know_edits2(word) or [word]
    def func(w):
        print(w)
        return NWORDS[w]
    return max(candidates, key=func)


d = correct('fucd')
d

fuck
fund


'fund'

In [86]:
# 错误引用
word = 'fucd'
w = know(edits1(word))
print(w)
NWORDS[w]



{'fuck', 'fund'}


TypeError: unhashable type: 'set'